In [1]:
import pandas as pd
import os
from enum import Enum
import json
from tqdm import tqdm
import math
from pathlib import Path
import random
import shutil
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
base_folder = os.path.join('.')

**SET PATH VARIABLES FOR TRAIN, TEST, AND VALIDATION FODLERS**

In [3]:
data_yaml_path = os.path.join(base_folder, 'data.yaml')
unsplitted_images_folder = os.path.join(base_folder, 'data_for_YOLO_training', 'unsplitted', 'images')
unsplitted_labels_folder = os.path.join(base_folder, 'data_for_YOLO_training', 'unsplitted', 'labels')
# Define paths to the output folders
output_folders = {
    'test': {
        'images': os.path.join(base_folder, 'data_for_YOLO_training', 'splitted', 'test', 'images'),
        'labels': os.path.join(base_folder, 'data_for_YOLO_training', 'splitted', 'test', 'labels')
    },
    'train': {
        'images': os.path.join(base_folder, 'data_for_YOLO_training', 'splitted', 'train', 'images'),
        'labels': os.path.join(base_folder, 'data_for_YOLO_training', 'splitted', 'train', 'labels')
    },
    'val': {
        'images': os.path.join(base_folder, 'data_for_YOLO_training', 'splitted', 'val', 'images'),
        'labels': os.path.join(base_folder, 'data_for_YOLO_training', 'splitted', 'val', 'labels')
    }
}

**CHECK TRAIN, TEST, AND VALIDATION FOLDERS**

In [ ]:
total_images = 0
total_labels = 0
for split in ['train', 'val', 'test']:
    image_stem_list = sorted([Path(n).stem for n in os.listdir(output_folders[split]['images']) if n.endswith('.jpg') and not n.startswith('.')])
    label_stem_list = sorted([Path(n).stem for n in os.listdir(output_folders[split]['labels']) if n.endswith('.txt') and not n.startswith('.')])
    n_images = len(image_stem_list)
    n_labels = len(label_stem_list)
    total_labels += n_labels
    total_images += n_images
    print(f"The folder {split}/images has {n_images} elements.")
    print(f"The folder {split}/labels has {n_labels} elements.")
    for image_stem in image_stem_list:
        if image_stem not in label_stem_list:
            print(f"Image {image_stem}.jpg is not labelled.")
    for label_stem in label_stem_list:
        if label_stem not in image_stem_list:
            print(f"Image {label_stem}.txt has no image.")

print(f"Total images: {total_images}")
print(f"Total labels: {total_labels}")

In [6]:
class Labels(Enum):
    gallina_bianca = 0
    gallina_rossa = 1

n_classes = 2
classes = ['gallina_bianca', 'gallina_rossa']

In [ ]:
!pip install ultralytics

In [ ]:
!pip install -U albumentations

In [ ]:
import torch
from ultralytics import YOLO

In [10]:
model = YOLO(os.path.join(base_folder, 'yolo11n.pt'))

In [ ]:
model.train(data=data_yaml_path, epochs=30)

In [ ]:
best_obj_detection_weights = os.path.join(base_folder, 'runs', 'detect', 'train', 'weights', 'best.pt')
model = YOLO(best_obj_detection_weights)
results = model.val()